In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Mar 24 17:52:34 2022

@author: asutter-adc
"""

import pandas as pd

dfile=pd.read_excel('Dealroom2011.xlsx')
n=len(dfile.columns.to_list())

dfile.insert(n, 'SIREN', '')

In [33]:
dfile

,id,NAME,PROFILE URL,WEBSITE,TAGLINE,ADDRESS,STREET,STREET NUMBER,STREET AND STREET NUMBER,ZIPCODE,...,TECHNOLOGIES,INCOME STREAMS,TECH STACK DATA (BY PREDICTLEADS),TRADE REGISTER NUMBER,SDGS,CORE SIDE VALUE,YEAR COMPANY BECAME UNICORN,YEAR COMPANY BECAME FUTURE UNICORN,PIC NUMBER,SIREN
0,79754,Bankin',https://app.dealroom.co/companies/bankin_,https://bankin.com/,None,"41, Rue de Prony, 75017 Paris, France",Rue de Prony,41,"Rue de Prony, 41",75017,...,mobile app,subscription,.net;aircall;android;amazon web services;azure...,"529196313 (Paris, France)",None,None,NaN,NaN,NaN,
1,2637820,Ron Dorff,https://app.dealroom.co/companies/ron_dorff,https://www.rondorff.com,Sportswear - Swimwear - Underwear – Ron Dorff,"5, Place André Malraux, 75001 Paris, France",Place André Malraux,5,"Place André Malraux, 5",75001,...,None,None,None,None,None,None,NaN,NaN,NaN,
2,1480887,Kyotherm,https://app.dealroom.co/companies/kyotherm,http://kyotherm.com,Kyotherm’s purpose is to provide project capit...,"Rue HalÃ©vy, 75009 Paris, France",Rue HalÃ©vy,None,Rue HalÃ©vy,75009,...,None,commission,apache;bootstrap;font awesome;google analytics...,None,affordable and clean energy (#7);climate actio...,core,NaN,2022.0,NaN,
3,1660484,Lemon Tri,https://app.dealroom.co/companies/lemon_tri,http://lemontri.fr,None,"Avenue Ã‰douard Vaillant, 93500 Pantin, France",Avenue Ã‰douard Vaillant,None,Avenue Ã‰douard Vaillant,93500,...,None,None,elementor;facebook;facebook pixel;flickity;fon...,"529759110 (Pantin, France)",None,None,NaN,NaN,NaN,
4,1479113,Eqinov,https://app.dealroom.co/companies/eqinov,http://eqinov.com,Demand side management operator,"Rue du Gouverneur-GÃ©nÃ©ral Ã‰bouÃ©, 92130 Iss...",Rue du Gouverneur-GÃ©nÃ©ral Ã‰bouÃ©,None,Rue du Gouverneur-GÃ©nÃ©ral Ã‰bouÃ©,92130,...,None,None,apache;font awesome;google analytics;google fo...,"804492726 (Issy-les-Moulineaux, France)",affordable and clean energy (#7);climate actio...,side,NaN,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
856,5741,Cinequant,https://app.dealroom.co/companies/cinequant,http://www.cinequant.com,Building statistical models and deploying them...,"Rue de Charenton, 75012 Paris, France",Rue de Charenton,None,Rue de Charenton,75012,...,None,subscription,nginx;jquery,None,None,None,NaN,NaN,NaN,
857,5177,APPARTINFO,https://app.dealroom.co/companies/appartinfo,http://www.appartinfo.com,APPARTINFO was founded in 2011,"Boulogne-Billancourt, France",None,None,None,None,...,None,None,cloudflare cdn;google analytics,None,None,None,NaN,NaN,NaN,
858,5006,Bipsly,https://app.dealroom.co/companies/bipsly,http://bipsly.com,Bipsly replaces the paper loyalty punchcards o...,"164, Boulevard Pereire, 75017 Paris, France",Boulevard Pereire,164,"Boulevard Pereire, 164",75017,...,None,subscription,None,None,None,None,NaN,NaN,NaN,
859,3104,statigram,https://app.dealroom.co/companies/statigram,http://www.statigram.com,Instagram power-users use Statigram every day ...,"Limoges, France",None,None,None,None,...,None,commission;advertising,None,None,None,None,NaN,NaN,NaN,


In [1]:

#nouvelle version des fonctions d'identification et de recherche de siren, prends en compte les siren attachés. Même process de recherche que test4.


from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
from re import compile


def is_luhn_valid(x: int) -> bool:
    """
    Application de la formule de Luhn à un nombre
    Permet la verification du numero SIREN et Siret d'un acheteur/etablissement
    Retour:
        - bool
    """
    try:
        luhn_corr = [0, 2, 4, 6, 8, 1, 3, 5, 7, 9]
        list_number_in_x = [int(i) for i in list(str(x))]
        l2 = [luhn_corr[i] if (index + 1) % 2 == 0 else i for index, i in enumerate(list_number_in_x[::-1])]
        if sum(l2) % 10 == 0:
            return True
        elif str(x)[:9] == "356000000":  # SIREN de la Poste
            if sum(list_number_in_x) % 5 == 0:
                return True
        return False
    except:
        return False



def isSIREN(c) :
    if len(c) == 11 :
        if c[0:3].isdigit() and c[3]==' ' and c[4:7].isdigit() and c[7]==' ' and c[8:].isdigit() :
            return is_luhn_valid(''.join(c.split()))

    elif len(c) == 9 :
        return is_luhn_valid(c)

    return False

def findSIREN(chaine):

    if len(chaine) < 14 :
        return 'Pas de SIREN trouvé'

    else :
        #première étape : on recherche dans le document un SIREN sous le type ??? ??? ???
        k=0

        while k <= len(chaine)-11 :
            if isSIREN(chaine[k:k+11]) :
                return chaine[k:k+11]

            k+=1

        #deuxième étape : on recherche un numéro de TVA au format FR??SIREN
        k= 0

        while k<=len(chaine) -13 :
            if chaine[k:k+2]=='FR' and chaine[k+2:k+13].isdigit() :
                if is_luhn_valid(chaine[k+4:k+13]):
                    return chaine[k+4:k+13]

            k+=1

        #troisième étape : on recherche un SIRET sans espace dont on extrait le SIREN, avec un espace le précédant (essentiel pour éviter les longues suites de chiffres présents dans les liens)
        k=0

        while k <= len(chaine)-16 :
            if chaine[k+1:k+15].isdigit() and chaine[k]==' ':
                if is_luhn_valid(chaine[k+1:k+15]) and is_luhn_valid(chaine[k+1:k+10]) :
                    return chaine[k+1:k+10]

            k+=1

        #quatrième étape : on cherche un SIREN tout attaché avec espaces avant et après
        k=0

        while k<= len(chaine)-11 :

            if chaine[k]==' ' and chaine[k+1:k+10].isdigit() and chaine[k+10]==' ':
                if is_luhn_valid(chaine[k+1:k+10]) :
                    return chaine[k+1:k+10]

            elif chaine[k]==' ' and chaine[k+1:k+10].isdigit() and chaine[k+10]=='.':
                if is_luhn_valid(chaine[k+1:k+10]) :
                    return chaine[k+1:k+10]

            k+=1


    return 'Pas de SIREN trouvé'




In [34]:
'''A RELANCER'''

lstpages = ['Mentions légales', 'Mentions Légales', 'mentions légales', 'Legal Notice', 'Legal Notices', 'Legal notice', 'Legal notices', 'Legal', 'Legals', 'legal', 'legals', 'Terms', 'terms', 'Conditions Générales de Vente', 'Conditions générales de vente', 'Conditions Générales', 'Conditions générales', 'conditions générales' ,'CG', 'Conditions générales', 'Conditions de vente', 'Conditions de Vente', 'Mentions légales & CGU', 'CGUV', 'CGU', 'Terms of use', 'Terms of Use', 'Legal Information', 'Legal information', 'Legal Informations', 'Legal informations', 'Privacy', 'privacy', 'Privacy Policy', 'Privacy policy', 'Terms of service', 'Legal Mentions', 'Legal mentions', 'legal mentions', 'Legal Terms', 'Legal terms', 'About Us', 'About us', 'Disclaimer']


for i in dfile.index.to_list() :
    print(i)
    url = dfile.loc[i, 'WEBSITE']
    try :
        if url !='' :

            if url[-1]=='/' :
                url=url[:-1]

            res = requests.get(url)

            soup=BeautifulSoup(res.content, 'lxml')

            k = 0
            length = len(lstpages)
            sirentrouve=False
            siren = 'Pages introuvables'
            pageouverte=False


            while k<length and not sirentrouve :

                page = lstpages[k]
                #premier test simple
                link = soup.find('a', string=page, href=True)

                if link is not None :
                    pageouverte=True
                    newurl = link.get('href')

                    if url[8:10]=='ww' :
                        test=url[12:-4]

                    else :
                        test = url[8:-4]

                    if newurl.find(test) == -1 and newurl[:2]!='ht' and newurl[:2]!='//' : #cas où url relatif
                        if url[-3:]=='/fr' and newurl[:3]=='/fr':
                            newurl = url+newurl[3:]

                        elif url[-4:]=='/fr/' and newurl[:3]=='/fr' :
                            newurl = url + newurl[4:]

                        else :
                            newurl = url + newurl

                    if newurl[:2]=='//': #cas où url non relatif mais ne commençant pas par http:
                        newurl='http:'+newurl

                    newres = requests.get(newurl)
                    contenu = str(newres.content)

                    #siren = findSIREN(contenu)

                    #sirentrouve = isSIREN(siren)

                    #if not sirentrouve : #gérons les espaces insécables

                    soup2=BeautifulSoup(newres.content, 'html.parser')
                    chaine2=str(soup2)

                    chaine3=chaine2.replace('\xa0', ' ')


                    siren = findSIREN(chaine3)
                    sirentrouve=isSIREN(siren)

                k+=1

            if not sirentrouve :

                link=None
                k=0
                while k<length and not sirentrouve :

                    page = lstpages[k]

                #on teste ceux où l'encodage a été correct mais où la string est à striper
                    if link is None :

                        liens = soup.find_all('a', href =True, string = True)
                        nbliens=len(liens)
                        compt=1
                        while compt <= nbliens and link is None :
                            tag=liens[nbliens-compt]
                            chaine=tag.string.strip()
                            if chaine.find(page)!=-1 :
                                link = tag

                            compt += 1

                #on teste ceux mal encodés, en misant sur le fait qu'ils sont encodés en latin-1
                    if link is None :

                        liens=soup.find_all('a', href=True, string=True)
                        nbliens=len(liens)
                        compt=1
                        while compt <= nbliens and link is None :
                            tag=liens[nbliens-compt]
                            chaine=str(tag.string).strip()
                            binaire= bytes(page, 'utf-8') #c'est la page qu'on décode
                            testl=binaire.decode('latin-1')

                            if chaine.find(testl)!=-1 : #on demande juste à ce que page se trouve dans string
                                link = tag

                            compt += 1



                    if link is not None :

                        newurl = link.get('href')

                        if url[8:10]=='ww' :
                            test=url[12:-4]

                        else :
                            test = url[8:-4]

                        if newurl.find(test) == -1 and newurl[:2]!='ht' and newurl[:2]!='//': #cas où url relatif
                            if url[-3:]=='/fr' and newurl[:3]=='/fr':
                                newurl = url+newurl[3:]

                            elif url[-4:]=='/fr/' and newurl[:3]=='/fr' :
                                newurl = url + newurl[4:]

                            else :
                                newurl = url + newurl

                        if newurl[:2]=='//':
                            newurl='http:'+newurl

                        newres = requests.get(newurl)
                        contenu = str(newres.content)

                        soup2=BeautifulSoup(newres.content, 'html.parser')
                        chaine2=str(soup2)
                        chaine3=chaine2.replace('\xa0', ' ')

                        siren = findSIREN(chaine3)
                        sirentrouve=isSIREN(siren)

                    k+=1

        #cas où le string contient des strings
            if not sirentrouve :

                link=None
                k=0
                while k<length and not sirentrouve :

                    page = lstpages[k]
                    if link is None :

                        liens = soup.find_all('a', href =True)
                        nbliens=len(liens)
                        compt=1
                        while compt <= nbliens and link is None :
                            tag=liens[nbliens-compt]
                            if page in tag.descendants :
                                link = tag

                            compt += 1

                    if link is None :

                        liens = soup.find_all('a', href =True)
                        nbliens=len(liens)
                        compt=1
                        while compt <= nbliens and link is None :
                            tag=liens[nbliens-compt]
                            for elem in tag.descendants :
                                if elem.string is not None :
                                    chaine = str(elem.string)
                                    if chaine.find(page)!=-1 :
                                        link = tag

                            compt += 1

                    if link is not None :

                        newurl = link.get('href')

                        if url[8:10]=='ww' :
                            test=url[12:-4]

                        else :
                            test = url[8:-4]

                        if newurl.find(test) == -1 and newurl[:2]!='ht' and newurl[:2]!='//': #cas où url relatif
                            if url[-3:]=='/fr' and newurl[:3]=='/fr':
                                newurl = url+newurl[3:]

                            elif url[-4:]=='/fr/' and newurl[:3]=='/fr' :
                                newurl = url + newurl[4:]

                            else :
                                newurl = url + newurl

                        if newurl[:2]=='//':
                            newurl='http:'+newurl

                        newres = requests.get(newurl)
                        contenu = str(newres.content)

                        #siren = findSIREN(contenu)

                        #sirentrouve = isSIREN(siren)

                        #if not sirentrouve : #gérons les espaces insécables

                        soup2=BeautifulSoup(newres.content, 'html.parser')
                        chaine2=str(soup2)
                        chaine3=chaine2.replace('\xa0', ' ')

                        siren = findSIREN(chaine3)
                        sirentrouve=isSIREN(siren)

                    k+=1
            if sirentrouve :
                dfile.loc[i, 'SIREN'] = siren
                
            else :
                pass

    except :
        pass




0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


151
152
153
154
155
156
157
158
159
160
161
162
163
164
165


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415


...

498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860


In [ ]:
##COMPLEMENT AVEC HOMONYMES

In [6]:
#construction des dictionnaires noms et nafs dont les clés sont les SIREN

i=0

noms={}
nafs={}
dates={}

siren1=open('basesiren1.txt', 'r', encoding='utf-8')
for line in siren1 :
  lst=line.strip().split(',')
  siren=lst[0]
  if lst[17]=='NAFRev2' :
    noms[siren]=lst[8]
    nafs[siren]=lst[16]
    dates[siren]=lst[1][:4]
siren1.close()

siren2=open('basesiren2.txt', 'r', encoding='utf-8')
for line in siren2 :
  lst=line.strip().split(',')
  siren=lst[0]
  if lst[17]=='NAFRev2' :
    noms[siren]=lst[8]
    nafs[siren]=lst[16]
    dates[siren]=lst[1][:4]
siren2.close()

siren3=open('basesiren3.txt', 'r', encoding='utf-8')
for line in siren3 :
  lst=line.strip().split(',')
  siren=lst[0]
  if lst[17]=='NAFRev2' :
    noms[siren]=lst[8]
    nafs[siren]=lst[16]
    dates[siren]=lst[1][:4]
siren3.close()

siren4=open('basesiren4.txt', 'r', encoding='utf-8')
for line in siren4 :
  lst=line.strip().split(',')
  siren=lst[0]
  if lst[17]=='NAFRev2' :
    noms[siren]=lst[8]
    nafs[siren]=lst[16]
    dates[siren]=lst[1][:4]
siren4.close()

siren5=open('basesiren5.txt', 'r', encoding='utf-8')
for line in siren5 :
  lst=line.strip().split(',')
  siren=lst[0]
  if lst[17]=='NAFRev2' :
    noms[siren]=lst[8]
    nafs[siren]=lst[16]
    dates[siren]=lst[1][:4]
siren5.close()



In [7]:
#lecture de la table NAF X tags DR et construction d'une matrice sous forme de dictionnaire
import pandas as pd
df=pd.read_excel('file2.xlsx')

df=df.set_index('Code NAF principal')

In [8]:
tags=df.columns.to_list()
NAF=df.index.to_list()

A={}
for tag in tags :
  for naf in NAF :
    A[(naf, tag)]=df.loc[naf, tag]

In [9]:
#on crée un dictionnaire où les clés sont les naf et les valeurs sont les listes ordonnées dans l'ordre décroissant de tags pour ce naf.

def insert(tag, naf, lst_tags): #insère un tag dans une liste de tags dont les A sont triés par ordre croissant
  for i in range(len(lst_tags)) :
    if A[(naf,tag)]>A[(naf,lst_tags[i])] :
      lst_tags.insert(i,tag)
      return

  lst_tags.append(tag)  

def tri(naf) :
  copy_tags=[]
  for tag in tags :
    insert(tag, naf, copy_tags)

  return copy_tags



tagspournaf={} #dictionnaire en question

for naf in NAF :
  tagspournaf[naf]=tri(naf)

In [10]:
from scipy.optimize import fmin 
from math import log

def param(naf) : #choix du paramètre de la loi de proba pour chaque naf
  lst_tags=tagspournaf[naf]
  n=len(lst_tags)
  N=0
  N2=0
  for k in range(n) :
    tag=lst_tags[k]
    N+=A[(naf,tag)]
    N2+=A[(naf,tag)]*k

  def fct(a) :
    return -N2*log(a) - N *log(1-a) + N*log(1-a**n)

  lst=[0.1, 0.3, 0.5, 0.7, 0.9]

  a="pas de paramètre optimal"
  for val in lst :
    try :
      a=fmin(fct,val)

    except :
      pass

  return a , N

dico_param={}
eff={}

for naf in NAF :
  dico_param[naf],eff[naf]=param(naf)

def f(naf,tag) :
  a=dico_param[naf]
  k=tagspournaf[naf].index(tag)

  if a == "pas de paramètre optimal" :
    return "NON"
    
  return a**k * (1-a)/(1-a**eff[naf])

Optimization terminated successfully.
         Current function value: 2.703367
         Iterations: 13
         Function evaluations: 26
Optimization terminated successfully.
         Current function value: 2.703367
         Iterations: 14
         Function evaluations: 28
Optimization terminated successfully.
         Current function value: 2.249341
         Iterations: 15
         Function evaluations: 30
Optimization terminated successfully.
         Current function value: 2.249341
         Iterations: 12
         Function evaluations: 24
Optimization terminated successfully.
         Current function value: 2.249341
         Iterations: 14
         Function evaluations: 28
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 7
         Function evaluations: 20
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 9
         Function evaluations: 26
Optimization terminated successfully

Optimization terminated successfully.
         Current function value: 15.746174
         Iterations: 13
         Function evaluations: 26
Optimization terminated successfully.
         Current function value: 15.746174
         Iterations: 11
         Function evaluations: 22
Optimization terminated successfully.
         Current function value: 15.746174
         Iterations: 14
         Function evaluations: 28
Optimization terminated successfully.
         Current function value: 15.746174
         Iterations: 15
         Function evaluations: 30
Optimization terminated successfully.
         Current function value: 62.361022
         Iterations: 18
         Function evaluations: 36
Optimization terminated successfully.
         Current function value: 62.361022
         Iterations: 15
         Function evaluations: 30
Optimization terminated successfully.
         Current function value: 62.361022
         Iterations: 9
         Function evaluations: 18
Optimization terminated succ

In [11]:
#score de compatibilité entre un code naf et une série de tags, fondés sur les données de l'échantillon de départ.

def newscore(naf, lst_tags) : #score de compatibilité entre un code naf et un ensemble de tags
  if naf not in NAF :
    return 0

  elif dico_param[naf] == "pas de paramètre optimal" :
    return 0 #à modifier

  else :
    produit = 1
    for tag in lst_tags :
      produit = produit * f(naf,tag) 

    return produit

In [12]:
#fonction qui prend un nom DR et sa série de tags, et cherche ses homonymes dans la base sirène. 
#calcule les scores de compatibilité avec tous les homonymes, et choisit celui présentant une compatibilité maximale. 



def newtrouvesiren(NOM, TAGS, annee) :
  siren_local = []
  for siren, nom in noms.items() :
    if nom==NOM and dates[siren]==annee :
      siren_local.append(siren)

  scoreinit=0
  res='indéterminable'
  for siren in siren_local :

    test=newscore(int(nafs[siren][:2]), TAGS)
    if test > scoreinit :
        scoreinit=test
        res=siren

 #     elif test == scoreinit :
 #       if date[siren][:4]==annee :
 #         scoreinit=test
 #         res=siren

    
  return res



In [36]:
for i in dfile.index.to_list() :
    if not isSIREN(dfile.loc[i, 'SIREN']):
        print(i)
        NOM=dfile.loc[i, 'NAME']
        TAGS = str(dfile.loc[i,'INDUSTRIES']).split(';')
        annee = dfile.loc[i,'LAUNCH DATE'][:4]
        siren = newtrouvesiren(NOM,TAGS,annee)
        if isSIREN(siren) :
            dfile.loc[i,'SIREN']=siren
        else :
            pass

782
783
784
785
786
787
788
789
790
791
793
794
795
796
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
827
828
829
831
832
833
836
837
838
839
840
841
842
843
844
846
847
848
849
850
851
852
853
854
855
857


/opt/conda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  import sys
/opt/conda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:39: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


858
859
860


In [37]:
dfile.to_excel('ultimateDR2011.xlsx')